# MultiHead Attention

By far the most important, and most significant component of the transformer is the Multihead Attention mechanism.
After all, the paper is titled, "Attention is all you need". From now on, Multihead Attention will be shorthanded to MHA from not on for convenience. 

Therefore, before diving deep into the rest of the transformer, I think it is worthwhile to try implementing 
this module from scratch, which in turn will help us understand ths subtle parts about MHA.

There does exist official PyTorch implementation, which is used for their official Transformer block, but I am a strong believer of
trying something from scratch to appreciate the details when using a de-facto implementation. 



# Attention
MHA is simply multiple Attention blocks stacked together, therefore in order to truly understand MHA, we need to do a deep dive on actual attention.
From the paper, Attention is defined with this mathematical equation:

$\text{Attention}(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$

with the following diagram to support the equation
![Attention Diagram](images/attention.jpeg)

# Q, K, and V
Q, K, and V stand for Query, Key, and Value respectively. To cite this awesome [Stack Overfow Answer](https://stats.stackexchange.com/questions/421935/what-exactly-are-keys-queries-and-values-in-attention-mechanisms)
> The key/value/query concepts come from retrieval systems. For example, when you type a query to search for some video on Youtube, the search engine will map your query against a set of keys (video title, description etc.) associated with candidate videos in the database, then present you the best matched videos (values).

One thing that really confused me personally when I was studying Attention is how they seemed to use the terms Q,K,V interchangeably even though they had different implications.
The following is what I mean:
![Attention Diagram 2](images/confusion.png "Attention and MHA")
The input to both Scaled-Dot-Product Attention and Multi-Head Attention are Q,K,V, but Multi-Head Attention has Scaled-Dot-Product Attention in them, and it seems to take in Q,K,V that is projected with Linear Layer.
Later on, they show the following equation for MHA that looks like the following:
![MHA Equation](images/mha_equation.png "MHA Equation")

Comparing it to the previos Attention equation which takes in Q,K,and V only, the Attention equation from MHA takes in 
$QW_{i}^{Q}$, $KW_{i}^{K}$, and $VW_{i}^{V}$. So which one is it...?


Maybe this was just me that was confused, but if you are like me who was also confused by this during the first read through of the paper, 
hopefully the following description may help.

The image for Attention Module is correct in the sense that **Q** (Query), **K** (Key), **V** (Value) are the inputs, but I think that MHA diagram should have something else as its inputs instead of **V,K,Q**. These values should instead be some other input, and the vector that went through **Linear** layer should be **Q,K,V** that goes into each Attention head.
I crudly redrew the Attention and MHA blocks to illustrate what I mean.
![New Attention Diagram](images/AttentionDiagram_New.png "New Attention Diagram for Illustration")

The Scaled Dot Product Attention box diagram would have the same internals as the previous diagram from the paper.

The new equation that follows the new diagram would be: 

![MHA Equation](images/new_mha_equation.png "MHA Equation")

$\text{MultiHead}(Q,K,V)=\text{Concat}(head_1,...,head_h)W^O$

$\text{where   } \text{head_i} = \text{Attention}(qW_i^Q, kvW_i^K, kvW_i^V)$

The **q** input gets turned into **Q** after being projected by $W^Q$, and **kv** gets turned into **K** and **V** after being projected by $W^K$ and $W^V$ respectively. **K** (Key), and **V** (Value) are extracted from the same inut vector. Personally, I am not sure why, but this is how it is in the paper. 

In self-attention (used in Encoder portion), **q** and **kv** are also equal, where as with non-self-attention (used in Decoder portion), **q** and **kv** are differen vectors.

# Coding Attention Module

Okay, so I think the general overview of the Attntion Module and MHA is somewhat complete. The rest of the detail will be understood better with code.

The first part of creating MHA module is to create the Scaled Dot Product Attention Module. MHA is a simple stacking of multiple Attention Modules, so once the actual Attention Module is properly formed, we can expect the rest to be pretty straight forward

In [4]:
# Import neccessary libraries
import torch
from torch import nn
from torch.nn import Module, ModuleList
from torch.nn import functional as F

In [5]:
class Attention(Module):
    def __init__(self):
        # initialize all the components here
        super().__init__()
        self.softmax = F.softmax()
    
    def forward(self, q,k,v):
        # q,k,v are in lower case latters to match Python practice. It is same as the capital letter variables from above
        k_T = torch.transpose(k) # need to transpose k to matrix multiply with q
        qkT = torch.mm(q,k_T)
        
        print(k.shape)
        d_k = k.shape
        # qkT_scaled = qkT * (1/())
        
        pass